### 영상 s3에 업로드

In [98]:
import logging
import boto3
from botocore.exceptions import ClientError

file_name = "./input_video/videoplayback.mp4"


def upload_file(file_name, bucket, object_name=None):
    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [105]:
s3 = boto3.client('s3')
with open(file_name, "rb") as f:
    s3.upload_fileobj(f, "kpmg-gobongbob", file_name[2:])

### 영상에서 text 추출

In [108]:
from __future__ import print_function
import time
import boto3
import pandas as pd

transcribe = boto3.client('transcribe')
job_uri = "https://kpmg-gobongbob.s3.us-east-2.amazonaws.com/input_video/"+file_name[14:]
print(job_uri)
job_name = job_uri[62:].rstrip(".mp4")

transcribe.start_transcription_job(
    TranscriptionJobName=job_name,
    Media={'MediaFileUri': job_uri},
    MediaFormat='mp4',
    LanguageCode='en-US'
)

while True:
    result = transcribe.get_transcription_job(TranscriptionJobName=job_name)
    if result['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
        break
    print("Not ready yet...")
    time.sleep(5)

if result['TranscriptionJob']['TranscriptionJobStatus'] == "COMPLETED":
    data = pd.read_json(result['TranscriptionJob']
                        ['Transcript']['TranscriptFileUri'])

print(data['results'][1][0]['transcript'])

https://kpmg-gobongbob.s3.us-east-2.amazonaws.com/input_video/videoplayback.mp4
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
this video is going to explain how to find inflection points on a graph. So let's suppose we have a graph. So here's a nice graph. This is the function f of T independent variable T and were asked to find What are the inflection points? Well, remember, inflection points are places where the con cavity changes, so it's a good idea to remember what Con cavity is. So for con cavity, we have con cave up where the graph is turning up or con cave down where the graph is turning down. So in order to figure out where the con cavity changes, we need to figure out where what the con cavity is and where it's con cave up and where it's Khan came down. So let's start from left to right. We started on th

In [258]:
items = data['results']["items"]

times = list(i["start_time"] for i in items if i["type"] != 'punctuation')
words = list(i['alternatives'][0]['content'] for i in items if i["type"] != 'punctuation')

#dict로 합치면 앞의 데이터가 날라감

### text에서 keyword 추출
- key idea: 연속적인 단어를 추출하기 위해서 첫번째 단어를 기준으로 하여 다음 리스트에 그 단어의 다음 단어가 존재하면 1을 반환, 아니면 0을 반환하고 break 한다. 
- 전체가 다 존재하면 간단하게 전체의 곱이 1이 될 것이고, 하나라도 충족하지 않으면, 0이 반환된다는 것에 중점을 맞춘다.

In [259]:
result = data['results'][1][0]['transcript']

In [260]:
comprehend = boto3.client('comprehend')

response = comprehend.detect_key_phrases(
    Text=result,
    LanguageCode='en'
)

results = list()
for i in range(10):
    results.append(response["KeyPhrases"][i]["Text"])
    
    
results = list(set(results))
print(results)

['a nice graph', 'T independent variable T', 'this video', 'the inflection points', 'places', 'inflection points', 'the function f', 'a graph']


In [261]:
finals = list()

for i in range(len(results)):
    finals.append([results[i]])
    result_list = results[i].split()
    for j in result_list:
        res_list = [k for k, value in enumerate(words) if value == j]
        finals[i].append(res_list)
        
print(finals)

[['a nice graph', [12, 19, 23, 59, 227, 295, 416], [24], [13, 20, 25, 79, 89]], ['T independent variable T', [32, 35], [33], [34], [32, 35]], ['this video', [0, 144, 148, 233, 329, 363, 389, 448, 566], [1]], ['the inflection points', [28, 43, 53, 78, 88, 100, 111, 137, 164, 187, 196, 199, 257, 274, 298, 326, 335, 345, 367, 383, 405, 420, 423, 427, 430, 436, 454, 461, 517, 522, 528, 531, 538, 549, 554, 559, 579, 582], [9, 44, 48, 165, 200, 445, 482, 492, 550, 575], [10, 45, 49, 515, 551, 576]], ['places', [51, 518, 555, 580]], ['inflection points', [9, 44, 48, 165, 200, 445, 482, 492, 550, 575], [10, 45, 49, 515, 551, 576]], ['the function f', [28, 43, 53, 78, 88, 100, 111, 137, 164, 187, 196, 199, 257, 274, 298, 326, 335, 345, 367, 383, 405, 420, 423, 427, 430, 436, 454, 461, 517, 522, 528, 531, 538, 549, 554, 559, 579, 582], [29, 145, 149], [30]], ['a graph', [12, 19, 23, 59, 227, 295, 416], [13, 20, 25, 79, 89]]]


In [262]:
is_straight = list([i[0]] for i in finals)
print(is_straight)

for i in range(len(finals)):
    only_num = finals[i][1:]
    for j in range(len(only_num[0])):
        is_straight[i].append([1])
        for k in range(1, len(only_num)):
            if only_num[0][j]+k in only_num[k]:
                is_straight[i][j+1].append(1)
            else:
                is_straight[i][j+1].append(0)
                break

print(is_straight)

[['a nice graph'], ['T independent variable T'], ['this video'], ['the inflection points'], ['places'], ['inflection points'], ['the function f'], ['a graph']]
[['a nice graph', [1, 0], [1, 0], [1, 1, 1], [1, 0], [1, 0], [1, 0], [1, 0]], ['T independent variable T', [1, 1, 1, 1], [1, 0]], ['this video', [1, 1], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0]], ['the inflection points', [1, 0], [1, 1, 1], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 1, 0], [1, 0], [1, 0], [1, 1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 1, 1], [1, 0], [1, 0], [1, 0], [1, 0]], ['places', [1], [1], [1], [1]], ['inflection points', [1, 1], [1, 1], [1, 1], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 1], [1, 1]], ['the function f', [1, 1, 1], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1

In [263]:
def multiple(num_list):
    result = 1
    for i in num_list:
        result *= i
    return result

In [264]:
is_straight_result = dict()
keys = list(i[0] for i in is_straight)
print(keys)

['a nice graph', 'T independent variable T', 'this video', 'the inflection points', 'places', 'inflection points', 'the function f', 'a graph']


In [265]:
values = list()

for i in range(len(is_straight)):
    keyword = is_straight[i]
    values.append([])
    for j in range(len(keyword[1:])):
        if multiple(keyword[j+1]):
            values[i].append(times[finals[i][1][j]])

print(values)

[['9.99'], ['11.96'], ['0.54'], ['16.34', '244.5'], ['21.4', '233.33', '246.69', '272.54'], ['3.29', '16.48', '18.81', '244.82', '258.04'], ['11.09'], ['4.71', '9.21']]


In [266]:
keyword_time = dict(zip(keys, values))
print(keyword_time)

{'a nice graph': ['9.99'], 'T independent variable T': ['11.96'], 'this video': ['0.54'], 'the inflection points': ['16.34', '244.5'], 'places': ['21.4', '233.33', '246.69', '272.54'], 'inflection points': ['3.29', '16.48', '18.81', '244.82', '258.04'], 'the function f': ['11.09'], 'a graph': ['4.71', '9.21']}


In [267]:
import json
json_val = json.dumps(keyword_time)
print(json_val)

{"a nice graph": ["9.99"], "T independent variable T": ["11.96"], "this video": ["0.54"], "the inflection points": ["16.34", "244.5"], "places": ["21.4", "233.33", "246.69", "272.54"], "inflection points": ["3.29", "16.48", "18.81", "244.82", "258.04"], "the function f": ["11.09"], "a graph": ["4.71", "9.21"]}
